In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import sys, importlib
from scripts.classes import Label, Sample
import scripts.util as util
importlib.reload(sys.modules['scripts.util'])

<module 'scripts.util' from 'c:\\Users\\Thai\\Desktop\\uSask\\CMPT318\\CMPT318Project\\src\\scripts\\util.py'>

# Data loading step

In [2]:
TRAIN_DATA = "../data/EuroSatData/train.csv"
TEST_DATA = "../data/EuroSatData/test.csv"
VALIDATION_DATA = "../data/EuroSatData/validation.csv"

# Load data into pandas dataframe from csv files
train_df = pd.read_csv(TRAIN_DATA)
train_df["Filename"] = "../data/EuroSatData/" + train_df["Filename"].astype(str)
test_df = pd.read_csv(TEST_DATA)
test_df["Filename"] = "../data/EuroSatData/" + test_df["Filename"].astype(str)
validation_df = pd.read_csv(VALIDATION_DATA)
validation_df["Filename"] = "../data/EuroSatData/" + validation_df["Filename"].astype(str)

In [3]:
# Load image data into new column called "Image"
train_df["Image"] = train_df["Filename"].apply(util.read_img)
test_df["Image"] = test_df["Filename"].apply(util.read_img)
validation_df["Image"] = validation_df["Filename"].apply(util.read_img)

In [4]:
# Prepare the x and y of each data set
x_train, y_train = util.pd_series_to_np_img(train_df["Image"]), train_df["Label"].values
x_test, y_test = util.pd_series_to_np_img(test_df["Image"]), test_df["Label"].values
x_validation, y_validation = util.pd_series_to_np_img(validation_df["Image"]), validation_df["Label"].values

# Model set up and training

In [5]:
pretrained_vgg16 = tf.keras.applications.vgg16.VGG16(include_top=False, input_shape=(64, 64, 3))

In [6]:
pretrained_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [7]:
model = tf.keras.Sequential()

In [8]:
# Ignore the last layer, which is the dense layers
for layer in pretrained_vgg16.layers:
  layer.trainable = False
  model.add(layer)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 16, 16, 256)       2

In [10]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=10))
model.add(tf.keras.layers.Activation("softmax"))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 16, 16, 256)       2

In [14]:
# lr_sched = tf.keras.optimizers.schedules.ExponentialDecay(
#   initial_learning_rate=0.001,
#   decay_steps=10000,
#   decay_rate=0.9
# )

model.compile(
  optimizer=tf.optimizers.Adam(learning_rate=0.0001),
  loss=tf.losses.SparseCategoricalCrossentropy(),
  metrics=["accuracy"]
)

In [15]:
# NOTE: Possible batch_size configuration
train_history = model.fit(x=x_train, y=y_train, validation_data=(x_validation, y_validation), epochs=10, verbose=2)

Epoch 1/10
591/591 - 159s - loss: 0.2811 - accuracy: 0.9097 - val_loss: 0.3809 - val_accuracy: 0.8722 - 159s/epoch - 269ms/step
Epoch 2/10
591/591 - 155s - loss: 0.2788 - accuracy: 0.9117 - val_loss: 0.3817 - val_accuracy: 0.8750 - 155s/epoch - 262ms/step
Epoch 3/10
591/591 - 154s - loss: 0.2774 - accuracy: 0.9117 - val_loss: 0.3801 - val_accuracy: 0.8722 - 154s/epoch - 261ms/step
Epoch 4/10
591/591 - 150s - loss: 0.2765 - accuracy: 0.9115 - val_loss: 0.3797 - val_accuracy: 0.8735 - 150s/epoch - 254ms/step
Epoch 5/10
591/591 - 150s - loss: 0.2756 - accuracy: 0.9118 - val_loss: 0.3788 - val_accuracy: 0.8746 - 150s/epoch - 254ms/step
Epoch 6/10
591/591 - 150s - loss: 0.2743 - accuracy: 0.9120 - val_loss: 0.3772 - val_accuracy: 0.8757 - 150s/epoch - 254ms/step
Epoch 7/10
591/591 - 152s - loss: 0.2732 - accuracy: 0.9133 - val_loss: 0.3759 - val_accuracy: 0.8765 - 152s/epoch - 257ms/step
Epoch 8/10
591/591 - 152s - loss: 0.2721 - accuracy: 0.9134 - val_loss: 0.3793 - val_accuracy: 0.8731 - 

In [166]:
# NOTE: Possible batch_size configuration
prediction = model.predict(x=x_test, verbose=2)
y_prediction = np.argmax(prediction, axis=1)

85/85 - 15s - 15s/epoch - 172ms/step


In [172]:
correct = y_test == y_prediction
accuracy = np.sum(correct)/2700
accuracy

0.8811111111111111